In [8]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import rasterio
import rasterio.mask
from matplotlib import colors, cm
from osgeo import gdal
import os
from rasterio.plot import show
from rasterio.plot import show_hist
from numpy import ma
# import richdem as rd
from pyspatialml import Raster
from tqdm import tqdm
import rioxarray as rxr
import xarray as xr
from shapely.ops import nearest_points

In [10]:
# Load geospatial datasets

train_data = gpd.read_file(r"./datasets/Train.gpkg",geometry='geometry')
test_data = gpd.read_file(r"./datasets/Test.gpkg",geometry='geometry')
val_data = gpd.read_file(r"./datasets/valtellina.gpkg",geometry='geometry')
road_network = gpd.read_file(r"./datasets/road_network.gpkg",geometry='geometry')
river_network = gpd.read_file(r"./datasets/river_network.gpkg",geometry='geometry')
fault_zones = gpd.read_file(r"./datasets/geological_faults.gpkg",geometry='geometry')
land_use = gpd.read_file(r"./datasets/land_use_land_cover.gpkg",geometry='geometry')



The purpose of this notebook is to process the test data

In [11]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open(r".\datasets\dtm.tif") as dtm_src:
    # Create empty lists to store the sampled values
    dtm_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        dtm_value = next(dtm_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        dtm_values.append(dtm_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['dtm'] = dtm_values
    


In [12]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open(r".\datasets\average_precipitation_2020.tif") as average_precipitation_src:
    # Create empty lists to store the sampled values
    average_precipitation_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        average_precipitation_value = next(average_precipitation_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        average_precipitation_values.append(average_precipitation_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['average_precipitation'] = average_precipitation_values


In [13]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open(r".\datasets\90_perc_precipitation_2020.tif") as perc_precipitation_src:
    # Create empty lists to store the sampled values
    perc_precipitation_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        perc_precipitation_value = next(perc_precipitation_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        perc_precipitation_values.append(perc_precipitation_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['perc_precipitation'] = perc_precipitation_values


In [14]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open("eastness.tif") as eastness_src:
    # Create empty lists to store the sampled values
    eastness_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        eastness_value = next(eastness_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        eastness_values.append(eastness_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['eastness'] = eastness_values


In [15]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open("northerness.tif") as northness_src:
    # Create empty lists to store the sampled values
    northness_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        northness_value = next(northness_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        northness_values.append(northness_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['northness'] = northness_values


In [16]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open("hillshade.tif") as hillshade_src:
    # Create empty lists to store the sampled values
    hillshade_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        hillshade_value = next(hillshade_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        hillshade_values.append(hillshade_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['hillshade'] = hillshade_values


In [17]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open("roughness.tif") as roughness_src:
    # Create empty lists to store the sampled values
    roughness_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        roughness_value = next(roughness_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        roughness_values.append(roughness_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['roughness'] = roughness_values


In [18]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open("slope_rad.tif") as slope_rad_src:
    # Create empty lists to store the sampled values
    slope_rad_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        slope_rad_value = next(slope_rad_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        slope_rad_values.append(slope_rad_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['slope_rad'] = slope_rad_values


In [19]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open("TPI.tif") as TPI_src:
    # Create empty lists to store the sampled values
    TPI_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        TPI_value = next(TPI_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        TPI_values.append(TPI_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['TPI'] = TPI_values


In [20]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open("TRI.tif") as TRI_src:
    # Create empty lists to store the sampled values
    TRI_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        TRI_value = next(TRI_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        TRI_values.append(TRI_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['TRI'] = TRI_values


In [21]:
# Open the DTM, TPI, and TWI raster files
with rasterio.open("aspect_degrees.tif") as aspect_degree_src:
    # Create empty lists to store the sampled values
    aspect_degree_values = []

    # Iterate through the multipolygons in your testing GeoDataFrame
    for geom in test_data['geometry']:
        # Calculate the centroid of the multipolygon
        centroid = geom.centroid
        
        # Sample values from each raster at the centroid
        lon, lat = centroid.x, centroid.y
        aspect_degree_value = next(aspect_degree_src.sample([(lon, lat)]))[0]

        # Append the sampled values to the lists
        aspect_degree_values.append(aspect_degree_value)

    # Add the sampled values as new columns in your testing GeoDataFrame
    test_data['aspect_degree'] = aspect_degree_values


In [22]:
test_data

,ID,geometry,dtm,average_precipitation,perc_precipitation,eastness,northness,hillshade,roughness,slope_rad,TPI,TRI,aspect_degree
0,ID_000001,POINT (541862.336 5103652.266),1303.143921,0.226309,0.335290,0.889065,0.457782,99,11.834473,0.726230,-0.289673,3.459900,62.755936
1,ID_000002,POINT (566456.496 5131798.978),2743.524658,0.175368,0.461412,0.726184,-0.687500,42,10.106689,0.622848,-0.038818,2.576111,133.432571
2,ID_000003,POINT (584598.972 5109016.391),1535.253906,0.194696,0.383039,-0.527985,0.849254,-10,8.464844,0.550582,0.034424,2.354599,328.130615
3,ID_000004,POINT (542414.162 5125941.301),1938.368408,0.170589,0.214643,-0.666372,0.745620,-8,9.158691,0.551983,-0.163818,2.562866,318.212341
4,ID_000005,POINT (532099.144 5133370.588),2013.937744,0.173891,0.294433,0.741497,-0.670956,1,21.929199,1.038145,-0.793945,6.329514,132.140900
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,ID_039996,POINT (605509.423 5146469.307),1204.146973,0.159315,0.377356,-0.965510,0.260364,-56,1.818970,0.132465,-0.249390,0.565750,285.091675
39996,ID_039997,POINT (526815.199 5147832.968),1891.869995,0.156467,0.242401,0.987688,0.156435,-75,0.000000,0.000000,0.000000,0.000000,-9999.000000
39997,ID_039998,POINT (526873.190 5147470.852),1891.869995,0.171749,0.286767,0.987688,0.156435,-75,0.000000,0.000000,0.000000,0.000000,-9999.000000
39998,ID_039999,POINT (569507.854 5101121.584),2015.124756,0.253332,0.526724,-0.378801,0.925478,-41,3.188599,0.239213,0.024658,0.962372,337.740601


In [23]:

# Create spatial indexes
test_data.sindex
road_network.sindex

# Create a function to calculate the nearest distance for a single geometry
def calculate_nearest_distance(test_geom):
    nearest_points_result = nearest_points(test_geom, road_network.unary_union)
    return nearest_points_result[0].distance(nearest_points_result[1])

# Use the apply method to calculate distances for all test geometries
test_data['distance_to_nearest_road'] = test_data['geometry'].apply(calculate_nearest_distance)


In [24]:
# Create spatial indexes
test_data.sindex
river_network.sindex

# Create a function to calculate the nearest distance for a single geometry
def calculate_nearest_distance(test_geom):
    nearest_points_result = nearest_points(test_geom, river_network.unary_union)
    return nearest_points_result[0].distance(nearest_points_result[1])

# Use the apply method to calculate distances for all test geometries
test_data['distance_to_nearest_river'] = test_data['geometry'].apply(calculate_nearest_distance)


In [25]:
# Create spatial indexes
test_data.sindex
fault_zones.sindex

# Create a function to calculate the nearest distance for a single geometry
def calculate_nearest_distance(test_geom):
    nearest_points_result = nearest_points(test_geom, fault_zones.unary_union)
    return nearest_points_result[0].distance(nearest_points_result[1])

# Use the apply method to calculate distances for all test geometries
test_data['distance_to_nearest_fault'] = test_data['geometry'].apply(calculate_nearest_distance)



In [26]:

# Calculate the centroid of each geometry
test_data['centroid'] = test_data['geometry'].centroid

# Extract latitude and longitude from the centroids
test_data['latitude'] = test_data['centroid'].y
test_data['longitude'] = test_data['centroid'].x
test_data['area'] = test_data['geometry'].area
test_data['perimeter'] = test_data['geometry'].length
test_data['bounding_box'] = test_data['geometry'].envelope
test_data['aspect_ratio'] = test_data['bounding_box'].apply(lambda geom: geom.bounds[2] / geom.bounds[3]) # Calculate aspect ratio
test_data['convex_hull'] = test_data['geometry'].convex_hull


In [27]:
test_data.head()

,ID,geometry,dtm,average_precipitation,perc_precipitation,eastness,northness,hillshade,roughness,slope_rad,...,distance_to_nearest_river,distance_to_nearest_fault,centroid,latitude,longitude,area,perimeter,bounding_box,aspect_ratio,convex_hull
0,ID_000001,POINT (541862.336 5103652.266),1303.143921,0.226309,0.335290,0.889065,0.457782,99,11.834473,0.726230,...,1000.917702,84.145427,POINT (541862.336 5103652.266),5.103652e+06,541862.335807,0.0,0.0,POINT (541862.336 5103652.266),0.106171,POINT (541862.336 5103652.266)
1,ID_000002,POINT (566456.496 5131798.978),2743.524658,0.175368,0.461412,0.726184,-0.687500,42,10.106689,0.622848,...,1649.903253,102.287223,POINT (566456.496 5131798.978),5.131799e+06,566456.495993,0.0,0.0,POINT (566456.496 5131798.978),0.110382,POINT (566456.496 5131798.978)
2,ID_000003,POINT (584598.972 5109016.391),1535.253906,0.194696,0.383039,-0.527985,0.849254,-10,8.464844,0.550582,...,584.534148,827.982355,POINT (584598.972 5109016.391),5.109016e+06,584598.971887,0.0,0.0,POINT (584598.972 5109016.391),0.114425,POINT (584598.972 5109016.391)
3,ID_000004,POINT (542414.162 5125941.301),1938.368408,0.170589,0.214643,-0.666372,0.745620,-8,9.158691,0.551983,...,436.181599,791.679670,POINT (542414.162 5125941.301),5.125941e+06,542414.162385,0.0,0.0,POINT (542414.162 5125941.301),0.105817,POINT (542414.162 5125941.301)
4,ID_000005,POINT (532099.144 5133370.588),2013.937744,0.173891,0.294433,0.741497,-0.670956,1,21.929199,1.038145,...,2311.669383,1813.340365,POINT (532099.144 5133370.588),5.133371e+06,532099.143919,0.0,0.0,POINT (532099.144 5133370.588),0.103655,POINT (532099.144 5133370.588)


In [28]:
test_data.to_csv(r'./datasets/processed_test.csv')